In [18]:
import os
import shutil
import pickle
import numpy as np
from pydub import AudioSegment
from pydub.generators import Triangle


def load_pickle(filepath):
    with open(filepath, "rb") as f:
        json_data = pickle.load(f)
    return json_data

In [10]:
a,b = 45,140

root_dir = "./tempo_estimation_output"
anchor_dir = os.path.join(root_dir, f"tempo_{a}_{b}", "eval_data", "multi")
file_path = os.path.join(anchor_dir, "anchor_zero_uni.pkl")

data = load_pickle(file_path)
ky = 'bothhand_y_bothfoot_y_torso_y_uni'

hit_idx = data[ky]["hit_idx"]
hit_ref_bpm = data[ky]["hit_ref_bpm"]
hit_seg_names = data[ky]["hit_seg_names"]
hit_anc_seq = data[ky]["hit_anc_seq"]      
hit_genres = data[ky]["hit_genres"]
filename = data[ky]["filename"]



In [23]:
video_dir = "./aist_dataset/video"

v_idx = 0  
fname = filename[v_idx].replace("cAll", "c01")
anc_seq = hit_anc_seq[v_idx].flatten()

video_path = os.path.join(video_dir, fname + ".mp4")
os.path.exists(video_path)

# load video with audio
from moviepy.editor import VideoFileClip
video = VideoFileClip(video_path)
duration = video.duration  # in seconds
print(f"Video duration: {duration} seconds")

Video duration: 8.74 seconds


In [ ]:

time = np.arange(len(anc_seq)) / 60


# beat_peaks_sec = time[anc_seq == 1]        # anc_seq is binary sequence with 1 at beat positions

click_duration = 50  # milliseconds
click_freq = 1200  # Hz

# Generate a single click sound
click = Triangle(click_freq).to_audio_segment(duration=click_duration)

onset_times = time[anc_seq == 1]  

total_duration = (len(anc_seq)/60)*1000  #  in milliseconds

audio = AudioSegment.silent(duration=total_duration)
for onset in onset_times:
    position = int(onset * 1000)  # Convert onset time to milliseconds
    audio = audio.overlay(click, position=position)

# Export the audio with clicks to a file
audio.export(os.path.join("./aist_dataset", f"{fname}_clicktrack.wav"), format="wav")
shutil.copy(video_path, os.path.join("./aist_dataset", f"{fname}.mp4"))


<_io.BufferedRandom name='./aist_dataset/gKR_sBM_c01_d28_mKR3_ch03_clicktrack.wav'>

'./aist_dataset/gKR_sBM_c01_d28_mKR3_ch03.mp4'